<a href="https://colab.research.google.com/github/Cheesykung/Sugar-cane-leaves-Recognition/blob/master/handcraft_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [475]:
#@title Import
import math
import cv2
import numpy as np
import sklearn.neighbors as sn
import skimage.feature as skf
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import random

# for debugging
# from google.colab.patches import cv2_imshow # for image display

In [ ]:
#@title Create Grey Model
train = pd.read_csv('/content/drive/MyDrive/imgTrans/Tr/train.csv') # edit path (csv only)
train.head()

featureTr = []
labelTr = []

# add label
train_label = train['label'].values
for label in train['label']:
  labelTr.append(label)

copy_label = labelTr.copy()
train_img = []

paraQuantize = 64
paraAngle = [0, 45, 90, 135]
paraDistance = [1]

# create model
for img_name in tqdm(train['id']):
  # read รูป
  path = '/content/drive/MyDrive/imgTrans/Tr/' + str(img_name) + '.JPG'; # path รูป
  pic = cv2.imread(path)

  imgArray = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY) # (รูปที่จะเปลี่ยน, สีที่เปลี่ยนไป) แปลงสีเป็นสีเทา
  img = (imgArray / (256/paraQuantize)).astype(int); # Image Quantization หยิบข้อมูลทุกๆ

  glcm = skf.greycomatrix(img, distances=paraDistance, angles=paraAngle, levels=paraQuantize, symmetric=True, normed=True)

  featureCon = skf.greycoprops(glcm, 'contrast')[0]
  featureEne = skf.greycoprops(glcm, 'energy')[0]
  featureHom = skf.greycoprops(glcm, 'homogeneity')[0]
  featureCor = skf.greycoprops(glcm, 'correlation')[0]
  featureTmp = np.hstack((featureCon, featureEne, featureHom, featureCor))
  featureTr.append(featureTmp)


featureTr = np.array(featureTr)

# save grey data
grey = featureTr.copy()

In [ ]:
#@title Create Color Histograms Model
train = pd.read_csv('/content/drive/MyDrive/imgTrans/Tr/train.csv') # edit path (csv only)
train.head()

train_img = [] # Reset Values
featureTr = [] # Reset Values

for img_name in tqdm(train['id']):
  # for _id in range(1,8):
  path = '/content/drive/MyDrive/imgTrans/Tr/' + str(img_name) + '.JPG';
  img = cv2.imread(path)
  train_img.append(img)
  # แปลงภาพให้อยู่บนปริภูมิสีHSV
  out = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  # แปลงข้อมูลจากเมตริกซ์ให้อยู่ในรูปแบบเวกเตอร์ เฉพาะ Hue
  out = out[:,:,0].reshape(1,-1);
  # สร้างฮิสโตแกรมจาก Hue
  hist, bins = np.histogram(out,bins = np.arange(0,256,1) )
  # Normalization เพื่อทำให้ Feature สามารถรองรับขนาดภาพที่แตกต่างกันได้
  featureTr.append([hist/np.sum(hist)])

featureTr = np.array(featureTr)
feature_Tr = featureTr.reshape(len(featureTr),-1)

# save color data
color = feature_Tr.copy()

In [478]:
#@title Default Run All

path = '/content/drive/MyDrive/imgTrans/Tr/green/DSC00031.JPG'; # edit picture path
img = cv2.imread(path)

# grey transform (test)
imgArray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
greyImg = (imgArray / (256/paraQuantize)).astype(int); # Image Quantization
glcm = skf.greycomatrix(greyImg, distances=paraDistance, angles=paraAngle, levels=paraQuantize, symmetric=True, normed=True)
featureCon = skf.greycoprops(glcm, 'contrast')[0]
featureEne = skf.greycoprops(glcm, 'energy')[0]
featureHom = skf.greycoprops(glcm, 'homogeneity')[0]
featureCor = skf.greycoprops(glcm, 'correlation')[0]
greyTs = [np.hstack((featureCon, featureEne, featureHom, featureCor))]

# color transform (test)
out = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
out = out[:,:,0].reshape(1, -1)
hist, bins = np.histogram(out,bins = np.arange(0, 256, 1))
tmp = [hist/np.sum(hist)]
colorTs = (np.array([tmp])).reshape(1, -1)

# concat 2d array grey + color (model)
featureTr = np.concatenate((grey, color), axis=1)
# concat 2d array grey + color (test)
featureTs = np.concatenate((greyTs, colorTs), axis=1)

classifier = sn.KNeighborsClassifier(n_neighbors=1)

# fit model
classifier.fit(featureTr, labelTr)

# prediction
out_color = classifier.predict(featureTs)

# calculate probrability
# x = classifier.predict_proba(featureTs)
# confident = math.floor(max(x[0].tolist()) * 100)
# x = classifier.score(grey, labelTr, sample_weight=None)

# Output
print('Answer is ' + str(out_color)) 

Answer is ['green']


In [ ]:
#@title Default Run (Grey)
#Testing Image Loader and Feature Extraction
path = '/content/drive/MyDrive/imgTrans/Tr/DSC00037.JPG'; # edit path
pic = cv2.imread(path)

imgArray = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
img = (imgArray / (256/paraQuantize)).astype(int); # Image Quantization

glcm = skf.greycomatrix(img, distances=paraDistance, angles=paraAngle, levels=paraQuantize, 
symmetric=True, normed=True)

featureCon = skf.greycoprops(glcm, 'contrast')[0]
featureEne = skf.greycoprops(glcm, 'energy')[0]
featureHom = skf.greycoprops(glcm, 'homogeneity')[0]
featureCor = skf.greycoprops(glcm, 'correlation')[0]
featureTs = [np.hstack((featureCon, featureEne, featureHom, featureCor))]

# define classifier
classifier = sn.KNeighborsClassifier(n_neighbors=1)

# learning
classifier.fit(grey, labelTr)

# predict
out = classifier.predict(featureTs)

# Output
print('Answer is ' + str(out))

In [ ]:
#@title Default Run (Color)
path = '/content/drive/MyDrive/imgTrans/Tr/green/DSC00031.JPG';
img = cv2.imread(path)
out = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# color transform (test)
out = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
out = out[:,:,0].reshape(1, -1)
hist, bins = np.histogram(out,bins = np.arange(0, 256, 1)) 
tmp = [hist/np.sum(hist)]
colorTs = (np.array([tmp])).reshape(1, -1)

# define classifier
classifier = sn.KNeighborsClassifier(n_neighbors=1)

# learning
classifier.fit(color, labelTr)

# predict
out_color = classifier.predict(colorTs)
print('Answer is ' + str(out_color))